In [9]:
import numpy as np
import pandas as pd
import textwrap
import nltk
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\SUKANYA
[nltk_data]     DAS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
df = pd.read_csv('bbc_text_cls.csv')

In [12]:
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [13]:
labels = set(df['labels'])
labels

{'business', 'entertainment', 'politics', 'sport', 'tech'}

In [14]:
label = 'business'

In [15]:
texts = df[df['labels'] == label]['text']
texts.head()

0    Ad sales boost Time Warner profit\n\nQuarterly...
1    Dollar gains on Greenspan speech\n\nThe dollar...
2    Yukos unit buyer faces loan claim\n\nThe owner...
3    High fuel prices hit BA's profits\n\nBritish A...
4    Pernod takeover talk lifts Domecq\n\nShares in...
Name: text, dtype: object

In [16]:
probs = {}

for doc in texts:
    lines = doc.split("\n")
    for line in lines:
        tokens = word_tokenize(line)
        for i in range(len(tokens) - 2):
            t_0 = tokens[i]
            t_1 = tokens[i + 1]
            t_2 = tokens[i + 2]
            key = (t_0, t_2)
            if key not in probs:
                probs[key] = {}
            if t_1 not in probs[key]:
                probs[key][t_1] = 1
            else:
                probs[key][t_1] += 1

In [17]:
for key, d in probs.items():
    total = sum(d.values())
    for k, v in d.items():
        d[k] = v / total

In [18]:
probs

{('Ad', 'boost'): {'sales': 1.0},
 ('sales', 'Time'): {'boost': 1.0},
 ('boost', 'Warner'): {'Time': 1.0},
 ('Time', 'profit'): {'Warner': 1.0},
 ('Quarterly', 'at'): {'profits': 1.0},
 ('profits', 'US'): {'at': 1.0},
 ('at', 'media'): {'US': 1.0},
 ('US', 'giant'): {'media': 0.1,
  'telecoms': 0.1,
  'banking': 0.2,
  'foods': 0.1,
  'retail': 0.1,
  'oil': 0.2,
  'mortgage': 0.1,
  'agrochemical': 0.1},
 ('media', 'TimeWarner'): {'giant': 1.0},
 ('giant', 'jumped'): {'TimeWarner': 1.0},
 ('TimeWarner', '76'): {'jumped': 1.0},
 ('jumped', '%'): {'76': 0.14285714285714285,
  '1.8': 0.14285714285714285,
  '11': 0.14285714285714285,
  '6': 0.14285714285714285,
  '10.7': 0.14285714285714285,
  '7': 0.14285714285714285,
  '22': 0.14285714285714285},
 ('76', 'to'): {'%': 1.0},
 ('%', '$'): {'to': 0.7727272727272727, 'at': 0.22727272727272727},
 ('to', '1.13bn'): {'$': 1.0},
 ('$', '('): {'1.13bn': 0.006802721088435374,
  '900m': 0.006802721088435374,
  '280bn': 0.02040816326530612,
  '86m':

In [19]:
texts.iloc[0].split("\n")

['Ad sales boost Time Warner profit',
 '',
 'Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.',
 '',
 'The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.',
 '',
 "Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers a

In [20]:
def spin_document(doc):
    lines = doc.split("\n")
    output = []
    for line in lines:
        if line:
            new_line = spin_line(line)
        else:
            new_line = line
        output.append(new_line)
    return "\n".join(output)            

In [21]:
detokenizer = TreebankWordDetokenizer()

In [22]:
texts.iloc[0].split("\n")[2]

'Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.'

In [23]:
detokenizer.detokenize(word_tokenize(texts.iloc[0].split("\n")[2]))

'Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.'

In [24]:
def sample_word(d):
    p0 = np.random.random()
    cumulative = 0
    for t, p in d.items():
        cumulative += p
        if p0 < cumulative:
            return t
    assert(False)

In [25]:
def spin_line(line):
  tokens = word_tokenize(line)
  i = 0
  output = [tokens[0]]
  while i < (len(tokens) - 2):
    t_0 = tokens[i]
    t_1 = tokens[i + 1]
    t_2 = tokens[i + 2]
    key = (t_0, t_2)
    p_dist = probs[key]
    if len(p_dist) > 1 and np.random.random() < 0.3:
      middle = sample_word(p_dist)
      output.append(t_1)
      output.append("<" + middle + ">")
      output.append(t_2)
      i += 2
    else:
      output.append(t_1)
      i += 1
  if i == len(tokens) - 2:
    output.append(tokens[-1])
  return detokenizer.detokenize(output)

In [26]:
np.random.seed(1234)

In [27]:
i = np.random.choice(texts.shape[0])
doc = texts.iloc[i]
new_doc = spin_document(doc)

In [28]:
print(textwrap.fill(
    new_doc, replace_whitespace=False, fix_sentence_endings=True))

Bombardier chief to leave company

Shares in train and plane-making
giant Bombardier have fallen to <to> a 10-year low following <against>
the departure <hands> of its chief executive and two members of the
<key> board.

Paul Tellier, who <which> was also Bombardier's
president <epicentre>, left the company amid an ongoing <£80m>
restructuring . Laurent Beaudoin, part of the family that controls the
Montreal-based firm, will take on <over> the role of CEO under a newly
created management structure . Analysts said <believe> the
resignations seem to have stemmed from a boardroom dispute . Under Mr
Tellier's tenure at the company <subsidy>, which began in January
<July> 2003, plans <according> to cut the worldwide workforce of
75,000 by almost <signing> a third <movement> by 2006 were announced .
The firm's snowmobile <auto> division and defence services unit were
also sold and Bombardier started the development <future> of a new
aircraft seating 110 to 135 passengers.

Mr Tellier had ind